In [488]:
# импорт библиотек
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt  
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
iris = datasets.load_iris()

## 3 Функции потерь и оптимизация
* подготовка данных
    * загрузка данных `iris`
    * В данных оставьте только 2 класса: Iris `Versicolor`, `Iris Virginica`.


### Загрузка данных и преобразование в `DataFrame`
* оставляем только `Versicolor`, `Iris Virginica`
* разделяем та тестовую и тренировочную выборку `20 / 80`

In [489]:
# Загружаем датасет с ирисами
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Фильтруем данные, оставляя только два класса: Versicolor и Virginica
df = df[df['target'].isin([1, 2])]
# Переименовываем колонки для удобства
df = df.rename(columns={
    'sepal length (cm)': 'sepal_length',
    'sepal width (cm)': 'sepal_width',
    'petal length (cm)': 'petal_length',
    'petal width (cm)': 'petal_width'
})
# Получаем данные для признаков (X) и целевой переменной (y)
df_X, df_y = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], df['target']

In [490]:
df['target'].value_counts()

target
1    50
2    50
Name: count, dtype: int64

In [491]:
df_X, df_y = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], df['target']

#### Стандартизация данных `x` должен оказаться в `0` `y` в 1

In [525]:
from sklearn. model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)
X_train.shape , y_train.shape

((80, 4), (80,))

In [526]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### `LogisticRegression` без библиотеки

In [541]:
from sklearn.metrics import accuracy_score # доля правильных 
import time 

class LogisticRegressionWithOutLib:
    def __init__(self, learning_rate, epoch, optimizer=None):
        """
        Инициализация логистической регрессии.

        Параметры:
        - learning_rate (float): Скорость обучения (learning rate).
        - epoch (int): Количество итераций градиентного спуска.
        - optimizer (str): Метод оптимизации: None (без оптимизации), 'RMSProp' или 'Nadam'.
        """
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.optimizer = optimizer
        self.weights = None
        self.bias = None
        self.gradient_sq = None
        self.mu = None

        # Parameters for RMSProp
        self.beta = 0.9
        self.epsilon = 1e-7

        # Parameters for Nadam
        self.beta1 = 0.9
        self.beta2 = 0.999

    def sigmoid(self, z):
        # Сигмоидная функция активации
        return 1 / (1 + np.exp(-z))

    def initialize_parameters(self, num_features):
        # Инициализация весов и смещения модели
        self.weights = np.zeros(num_features)
        self.bias = 0
        self.gradient_sq = np.zeros(num_features)
        self.mu = np.zeros(num_features)

    def compute_cost(self, y, y_pred):
        # Add epsilon to prevent divide by zero and log of zero
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        
        # Compute the log loss
        return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def rmsprop_update(self, dw):
        # Обновление параметров с использованием метода RMSProp
        self.gradient_sq = self.beta * self.gradient_sq + (1 - self.beta) * dw**2
        self.weights -= (self.learning_rate / (np.sqrt(self.gradient_sq) + self.epsilon)) * dw

    def nadam_update(self, dw):
    # Обновление параметров с использованием метода Nadam
        self.mu = self.beta1 * self.mu + (1 - self.beta1) * dw
        self.gradient_sq = self.beta2 * self.gradient_sq + (1 - self.beta2) * dw**2
        mu_hat = self.mu / (1 - self.beta1)
        grad_hat = dw / (1 - self.beta2)
        self.weights -= self.learning_rate * (mu_hat / (np.sqrt(self.gradient_sq) + self.epsilon) + self.beta1 * grad_hat / (1 - self.beta1))

    def gradient_descent(self, X, y):
        num_samples, num_features = X.shape
        self.initialize_parameters(num_features)

        for _ in range(self.epoch):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)

            # Заменяем значения y_pred, чтобы избежать точного совпадения с 0 или 1
            epsilon = 1e-15
            y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

            cost = self.compute_cost(y, y_pred)

            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)

            if self.optimizer == 'RMSProp':
                self.rmsprop_update(dw)
            elif self.optimizer == 'Nadam':
                self.nadam_update(dw)
            else:
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

            # Дополнительно выводим значение стоимости на каждой итерации
            if _ % 100 == 0:
                print(f"Iteration {_}: Cost {cost}")

    def fit(self, X, y):
        # Обучение модели
        self.gradient_descent(X, y)

    def predict(self, X):
        # Предсказание классов для новых данных
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(linear_model)
        y_pred_cls = [1 if i > 0.5 else 0 for i in y_pred]
        return y_pred_cls

def comparison_different_method(
        X_train, X_test, y_train, y_test,
        learning_rate = 1, epoch= 100
        ):
    # Create models, one without optimization and two with optimization (RMSProp and Nadam)
    results_df = None
    for model, method_name in [
        (LogisticRegressionWithOutLib(
            learning_rate=learning_rate,
            epoch=epoch,
            optimizer=None),
            "Без оптимизации"),
        (LogisticRegressionWithOutLib(
            learning_rate=learning_rate,
            epoch=epoch,
            optimizer='RMSProp'),
            "С RMSProp"),
        (LogisticRegressionWithOutLib(
            learning_rate=learning_rate,
            epoch=epoch,
            optimizer='Nadam'),
            "С Nadam")]:

        start_time = time.time()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        end_time = time.time()
        execution_time = end_time - start_time

        # Create a DataFrame for the current result
        result_df = pd.DataFrame([
            [method_name, accuracy, execution_time, learning_rate, epoch]],
            columns=["Метод", "Метрика", "Время работы", "learning_rate", "epoch"])

        # Concatenate the current result DataFrame with the results_df
        results_df = pd.concat([results_df, result_df], ignore_index=True)

    return results_df

results_df = comparison_different_method(
    X_train, X_test, y_train, y_test, 0.001, 1000)
results_df

Iteration 0: Cost 0.6931471805599453
Iteration 100: Cost 0.5529628332271666
Iteration 200: Cost 0.422281317782116
Iteration 300: Cost 0.30001799230527515
Iteration 400: Cost 0.18513793768447992
Iteration 500: Cost 0.07669983673686827
Iteration 600: Cost -0.026127092799949226
Iteration 700: Cost -0.12406299318627438
Iteration 800: Cost -0.21772603755166814
Iteration 900: Cost -0.30764462758637157
Iteration 0: Cost 0.6931471805599453
Iteration 100: Cost 0.5842214577742839
Iteration 200: Cost 0.5107988570412753
Iteration 300: Cost 0.45322277779240794
Iteration 400: Cost 0.40480882545861674
Iteration 500: Cost 0.36422027777815574
Iteration 600: Cost 0.330160882554304
Iteration 700: Cost 0.30121612882307686
Iteration 800: Cost 0.27544048259853326
Iteration 900: Cost 0.25235394835035674
Iteration 0: Cost 0.6931471805599453
Iteration 100: Cost 0.01828898977603686
Iteration 200: Cost 0.014767335265960569
Iteration 300: Cost 0.01304152272052761
Iteration 400: Cost 0.011961695941376239
Iteration

,Метод,Метрика,Время работы,learning_rate,epoch
0,Без оптимизации,0.35,0.859640,0.001,1000
1,С RMSProp,0.15,0.843041,0.001,1000
2,С Nadam,0.05,0.860637,0.001,1000


In [533]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
test_accuracy = logistic_regression.score(X_test, y_test)
print("Точность на тестовом наборе:", test_accuracy)


Точность на тестовом наборе: 0.8


$$Accuracy=\frac{Общее\ количество\ предсказаний}{Количество\  правильных\  предсказаний}$$
